In [ ]:
from google.colab import drive

drive.mount("//content//gdrive")

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import models,layers,Sequential
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

In [2]:
brainPath=r"C:\Users\matrix\Desktop\brainData"

In [3]:
import pickle
fileOpen = open(brainPath+'/brain_data',"rb")
data = pickle.load(fileOpen)
fileOpen.close()

trainData=data['X']
trainDataLables=data['y']
print(trainData.shape,trainDataLables.shape)

(600, 256, 256) (600, 2)


In [4]:
input_shape=(256,256)

In [5]:
def conv_block(filters):
    block = Sequential([
        layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPool2D()
    ]
    )
    
    return block

In [6]:
def dense_block(units, dropout_rate):
    block = Sequential([
        layers.Dense(units, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate)
    ])
    
    return block

In [ ]:
model = models.Sequential([
    layers.Conv2D(filters=16,kernel_size=7,kernel_initializer='he_uniform',activation='relu',input_shape=input_shape),
    layers.Conv2D(filters=16,kernel_size=7,kernel_initializer='he_uniform',activation='relu'),
    layers.MaxPooling2D(),


    conv_block(32),
    conv_block(64),
        
    conv_block(128),
    tf.keras.layers.Dropout(0.2),
    
    conv_block(256),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    dense_block(512, 0.5),
    dense_block(128, 0.3),
    dense_block(64, 0.1),
    layers.Dense(2, activation='sigmoid')
])
model.summary()

In [ ]:
METRICS = [tf.keras.metrics.AUC(name='auc')]
model.compile(
        optimizer='adam',
        loss=tf.keras.losses.binary_crossentropy,
        metrics=METRICS
    )

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("brainModel.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(trainData, trainDataLables, test_size=0.25, random_state=42)

In [ ]:
history = model.fit(
    x_train,y_train,
    # validation_data=(x_val,y_val),
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
    epochs=10,verbose=2
)